# Projeto 2 - Classificador Automático de Sentimento

**Nome:** Beatriz R. Bernardino. <br>
**Nome:** Vitor I. B. Bandeira.

In [410]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [411]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
from IPython.display import display
import re 

In [412]:
planilha1=pd.read_excel("era uma vez em hollywood.xlsx", sheet_name='Treinamento')
#Retirando os rt das planilhas
excel1=planilha1.loc[planilha1['Classificação']!=3]

planilha2=pd.read_excel("era uma vez em hollywood2.xlsx", sheet_name='Treinamento')
#Jutando as planilhas para analise dos dados
tabela = excel1.append(planilha2)

In [413]:
#Funçaõ para CleanUp dos caracteres indesejados 
def cleanup(text):
    novoindex=[]
    punctuation = '[!\" \n-.:?;,@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    for frase in text:
         novoindex.append(re.sub(pattern, ' ', frase))
    return novoindex
   

In [414]:
#Aplicando a função CleanUp na nossa planilha analisada
tabela_limpa=cleanup(tabela.Treinamento)
#Ajustando tabela para análise dos dados
tabela_correta=tabela.drop(columns="Treinamento")
tabela_correta['tweets']=tabela_limpa
tabela_limpa

[' frckribery recebi hoje e só lembrei de certas pessoas vendo era uma vez em hollywood  ou melhor  perdendo',
 'que no caso são  bacurau  era uma vez em hollywood  it capítulo 2  brinquedo assassino  aquele trash de lei   e yesterday  mas não sei se conseguirei ver tudo  tá tudo bem também ',
 ' eduu_miranda era uma vez gore em hollywood',
 'eu quero muito assistir bacurau e era uma vez em hollywood mas ai cara pq cinema tem que ser tão caro    ',
 'vai sair uma versão de 4 horas de era uma vez    em hollywood  e meudeus eu precisook',
 'bacurau dá de 10 x 0 em era uma vez em hollywood   achei o filme do tarantino meio decepcionante ',
 'quanto mais leio/ouço sobre  era uma vez em    hollywood  mais eu fico revoltado com a necessidade de background que o filme tem',
 'boa noite seguidores  hoje a crítica vai para o filme era uma vez    em hollywood  nota 7 5',
 ' era uma vez em hollywood  ultrapassa a marca dos us  300 milhões mundialmente   https //t co/e6tivo9bav https //t co/vygrqz

## Tabela: 0, 1, 2. - Todos

In [415]:
#Aqui foi feito o calculo das tabelas de frequência absoluta e relativa para todos os nosso tweets.
tabela_correta

,Classificação,tweets
0,2,frckribery recebi hoje e só lembrei de certas...
1,0,que no caso são bacurau era uma vez em holly...
2,0,eduu_miranda era uma vez gore em hollywood
3,0,eu quero muito assistir bacurau e era uma vez ...
5,1,vai sair uma versão de 4 horas de era uma vez ...
6,2,bacurau dá de 10 x 0 em era uma vez em hollywo...
9,2,quanto mais leio/ouço sobre era uma vez em ...
10,1,boa noite seguidores hoje a crítica vai para ...
11,0,era uma vez em hollywood ultrapassa a marca ...
12,0,dá rt se hoje você tá feliz tá animado tá an...


#### Frequências absolutas

In [416]:
#transformando as informações da tabelas em strings
tabela_str_total = tabela_correta.tweets.to_string()

limpo_split = pd.Series(tabela_str_total.split())
#obtendo a frequência absoluta
tabela_freq_abs_total = limpo_split.value_counts()
tabela_freq_abs_total
total_valores=len(tabela_freq_abs_total)

#### Frequências relativas

In [417]:
#Obtendo a frequência relativa
tabela_freq_rel_total = limpo_split.value_counts(True)
tabela_freq_rel_total

uma               0.069408
era               0.068092
vez               0.063158
em                0.061184
hollywood         0.041776
de                0.018092
...               0.018092
que               0.016447
e                 0.013487
eu                0.012500
o                 0.012171
assistir          0.010197
no                0.009211
ver               0.009211
não               0.009211
é                 0.009211
a                 0.008553
um                0.006908
bacurau           0.006908
do                0.005921
filme             0.005921
assisti           0.005592
vi                0.004934
cinema            0.004934
vou               0.004934
e...              0.004276
pra               0.004276
mais              0.004276
ontem             0.003618
quero             0.003289
                    ...   
premiado          0.000329
perdi             0.000329
277               0.000329
danado            0.000329
257               0.000329
carlos_history    0.000329
t

## Tabela: 1, 2 - Relevantes

In [418]:
#Aqui separamos a tabela principal em apenas tweets relevantes, ou seja, classificados em 1 e 2.
excel_relevantes=tabela_correta.loc[tabela_correta['Classificação']!=0]
excel_relevantes

,Classificação,tweets
0,2,frckribery recebi hoje e só lembrei de certas...
5,1,vai sair uma versão de 4 horas de era uma vez ...
6,2,bacurau dá de 10 x 0 em era uma vez em hollywo...
9,2,quanto mais leio/ouço sobre era uma vez em ...
10,1,boa noite seguidores hoje a crítica vai para ...
17,1,eu só queria registrar que vi era uma vez em h...
20,2,quase dormi assistindo era uma vez em hollywood
26,1,era uma vez em hollywood é muito bom gente pra...
27,1,assisti era uma vez em hollywood e ainda e...
30,1,era uma vez em hollywood é um puta filme do...


## Tabela de frequência - Bom

#### Frequências absolutas

In [419]:
#Separando a tabela do em '1' (bom) para  obter a frequência absoluta dos tweets
excel_bom=excel_relevantes.loc[excel_relevantes['Classificação']==1]
excel_bom

,Classificação,tweets
5,1,vai sair uma versão de 4 horas de era uma vez ...
10,1,boa noite seguidores hoje a crítica vai para ...
17,1,eu só queria registrar que vi era uma vez em h...
26,1,era uma vez em hollywood é muito bom gente pra...
27,1,assisti era uma vez em hollywood e ainda e...
30,1,era uma vez em hollywood é um puta filme do...
36,1,era uma vez em hollywood = dahora cena final s...
41,1,nem vi a derrota do gremio fui ver era uma v...
49,1,interessante que quanto mais eu reflito sobre ...
55,1,surtemprala 🤦🏼‍♀️ meu deus só um dos melhore...


In [420]:
#Transformando em string
tabela_str_bom = excel_bom.tweets.to_string()
limpo_split = pd.Series(tabela_str_bom.split())
#Obtendo tabela absoluta
tabela_freq_abs_bom = limpo_split.value_counts()
tabela_freq_abs_bom
# x=len(tabela_freq_abs_bom)


uma             65
era             61
vez             60
em              60
hollywood       45
...             17
que             16
de              15
o               15
é               14
e               13
eu              12
vi               9
assisti          7
bom              7
um               7
filme            7
a                6
sobre            5
do               5
não              5
só               5
assistir         4
ontem            4
bacurau          4
novo             4
ver              4
mais             3
foi              3
final            3
                ..
49               1
conforme         1
três             1
8/10             1
barco            1
16               1
26               1
atrasada         1
manow            1
demais           1
amadeus          1
derrota          1
boa              1
se               1
sa...            1
223              1
mês              1
prometido        1
caralhos         1
aaugustiado      1
b...             1
hollywo...  

#### Frequências relativas

In [421]:
#Obtendo a frequência relativa bom
tabela_freq_rel_bom = limpo_split.value_counts(True)
tabela_freq_rel_bom

uma             0.077289
era             0.072533
vez             0.071344
em              0.071344
hollywood       0.053508
...             0.020214
que             0.019025
de              0.017836
o               0.017836
é               0.016647
e               0.015458
eu              0.014269
vi              0.010702
assisti         0.008323
bom             0.008323
um              0.008323
filme           0.008323
a               0.007134
sobre           0.005945
do              0.005945
não             0.005945
só              0.005945
assistir        0.004756
ontem           0.004756
bacurau         0.004756
novo            0.004756
ver             0.004756
mais            0.003567
foi             0.003567
final           0.003567
                  ...   
49              0.001189
conforme        0.001189
três            0.001189
8/10            0.001189
barco           0.001189
16              0.001189
26              0.001189
atrasada        0.001189
manow           0.001189


In [422]:
#Total de palavras boas
total_bom=tabela_freq_abs_bom.sum()

## Tabela de frequência - Ruim

In [423]:
#Obtendo da tabela total os valores ruins (2)
excel_ruim=tabela_correta.loc[tabela['Classificação']==2]
excel_ruim

,Classificação,tweets
0,2,frckribery recebi hoje e só lembrei de certas...
6,2,bacurau dá de 10 x 0 em era uma vez em hollywo...
9,2,quanto mais leio/ouço sobre era uma vez em ...
20,2,quase dormi assistindo era uma vez em hollywood
31,2,josiasteofilo não consegui assistir o era uma...
86,2,ai eu achei era uma vez em hollywood decepcion...
87,2,as pessoas podiam fazer crítica de cinema tamb...
93,2,sobre era uma vez em hollywood de tarant...
95,2,assisti era uma vez em hollywood ontem e que f...
100,2,fazia tempo que eu não via um filme tão ruim q...


#### Frequências absolutas

In [424]:
#transformando as informações da tabelas em strings
tabela_str_ruim = excel_ruim.tweets.to_string()
limpo_split = pd.Series(tabela_str_ruim.split())
#obtendo a frequência absoluta
tabela_freq_abs_ruim = limpo_split.value_counts()
total_ruim=tabela_freq_abs_ruim.sum()

#### Frequências relativa

In [425]:
#Obtendo tabela de frequência relativa
tabela_freq_rel_ruim = limpo_split.value_counts(True)
tabela_freq_rel_ruim

era               0.080645
uma               0.077419
em                0.074194
vez               0.074194
hollywood         0.051613
de                0.029032
não               0.019355
e                 0.016129
que               0.016129
assisti           0.016129
o                 0.016129
eu                0.012903
...               0.012903
filme             0.012903
ontem             0.012903
se                0.009677
sobre             0.009677
uma...            0.009677
é                 0.009677
ruim              0.006452
e...              0.006452
assistindo        0.006452
achei             0.006452
0                 0.006452
a                 0.006452
um                0.006452
mais              0.006452
mal               0.006452
m...              0.003226
hollywood”        0.003226
                    ...   
58                0.003226
bacurau           0.003226
kmendoncafilho    0.003226
191               0.003226
um...             0.003226
certas...         0.003226
1

## Tabela: 0 - Irrelevantes

In [426]:
#Obtendo da tabela total os valores irrelevantes (0)
excel_irrelevantes=tabela_correta.loc[tabela_correta['Classificação']==0]
excel_irrelevantes

,Classificação,tweets
1,0,que no caso são bacurau era uma vez em holly...
2,0,eduu_miranda era uma vez gore em hollywood
3,0,eu quero muito assistir bacurau e era uma vez ...
11,0,era uma vez em hollywood ultrapassa a marca ...
12,0,dá rt se hoje você tá feliz tá animado tá an...
15,0,a fila tá enorme esquece vou ver era uma vez...
16,0,nao ta passando mais bacurau nao ta passando m...
19,0,ontem eu descobri assistindo era uma vez e...
21,0,bixo faz uns 6 meses que não vou no cinema po...
22,0,quem eh que vamo assisti bacurau e it se era ...


#### Frequências absolutas

In [427]:
#obtendo a frequência absoluta
limpo_split = pd.Series(tabela_str_irrelevantes.split())
tabela_freq_abs_irrelevantes = limpo_split.value_counts()
tabela_freq_abs_irrelevantes

uma              122
era              121
vez              109
em               103
hollywood         66
...               34
de                31
que               29
assistir          26
ver               24
no                24
e                 23
eu                22
a                 18
não               17
o                 17
bacurau           16
vou               14
do                13
cinema            13
um                12
é                 11
pra               11
e...              10
it                 9
quero              9
q                  9
2                  9
holly...           8
vai                8
                ... 
55                 1
torrent            1
oq                 1
quis               1
até                1
amg                1
utilidade          1
passado            1
lolaescreva        1
drink              1
105                1
rm                 1
proje...           1
confirmou          1
44                 1
meses              1
282          

#### Frequências relativas

In [428]:
#Obtendo frequências relativas
tabela_freq_rel_irrelevantes = limpo_split.value_counts(True)
tabela_freq_rel_irrelevantes

uma              0.064584
era              0.064055
vez              0.057702
em               0.054526
hollywood        0.034939
...              0.017999
de               0.016411
que              0.015352
assistir         0.013764
ver              0.012705
no               0.012705
e                0.012176
eu               0.011646
a                0.009529
não              0.008999
o                0.008999
bacurau          0.008470
vou              0.007411
do               0.006882
cinema           0.006882
um               0.006353
é                0.005823
pra              0.005823
e...             0.005294
it               0.004764
quero            0.004764
q                0.004764
2                0.004764
holly...         0.004235
vai              0.004235
                   ...   
55               0.000529
torrent          0.000529
oq               0.000529
quis             0.000529
até              0.000529
amg              0.000529
utilidade        0.000529
passado     

## Implementação Naive Bayes

In [429]:
#Realizando Laplace smoothing no código do Naive Bayes
class_relev={}
for frase1 in tabela_limpa:
    frase = frase1.split()
    log_bom = 0
    log_ruim=0
    log_irre=0
    for p in frase:
        if p in tabela_freq_rel_bom:
            log_bom += np.log10(tabela_freq_rel_bom[p])
        if p in tabela_freq_rel_ruim:
            log_ruim += np.log10(tabela_freq_rel_ruim[p])
        if p in tabela_freq_rel_irrelevantes:
            log_irre += np.log10(tabela_freq_rel_irrelevantes[p])
    log_bom*=-1
    log_ruim*=-1
    log_irre*=-1
    if log_bom > log_ruim and log_bom > log_irre:
        class_relev[frase1]=1
    elif log_ruim > log_bom and log_ruim > log_irre:
        class_relev[frase1]=2
    else:
        class_relev[frase1]=0


class_relevt = pd.DataFrame.from_dict(class_relev,orient='index')
class_relevt

tabela_correta

naive=[]
for i in class_relevt[0]:
    naive.append(i)
naive

tabela_correta["naive"]=naive    
classificacao=[]
for i in tabela_correta.Classificação:
    classificacao.append(i)

#Classificando nossos resultados como: Verdadeiro Irrelevante, Falso Positivo, Falso Negativo, Verdadeiro Positivo, 
#Falso Irrelevante, Falso Negativo, Verdadeiro Negativo, Falso Irrelevante e Falso Positivo.
tipo=[0]*301
i=0
for numero in classificacao:
    if numero==naive[i] and numero==0:
        tipo[i]="Verdadeiro Irrelevante"
        
    elif numero!=naive[i] and numero==0:
        if  numero!=naive[i] and naive[i]==1:
            tipo[i]="Falso Positivo"
        elif numero!=naive[i] and naive[i]==2:
            tipo[i]= 'Falso Negativo'
            
    elif numero==naive[i] and numero==1:
            tipo[i]="Verdadeiro Positivo"
        
    elif numero!=naive[i] and numero==1:
        if naive[i]==0:
            tipo[i]="Falso Irrelevante"
        elif naive[i]==2:
            tipo[i]="Falso Negativo"
        
    elif numero==naive[i] and numero==2:
        tipo[i]="Verdadeiro Negativo"
        
    elif numero!=naive[i] and numero==2:
        if naive[i]==0:
            tipo[i]="Falso Irrelevante"
        elif naive[i]==1:
            tipo[i]="Falso Positivo"
    i+=1



tabela_correta["Tipo"]=tipo  
tabela_correta

,Classificação,tweets,naive,Tipo
0,2,frckribery recebi hoje e só lembrei de certas...,0,Falso Irrelevante
1,0,que no caso são bacurau era uma vez em holly...,0,Verdadeiro Irrelevante
2,0,eduu_miranda era uma vez gore em hollywood,0,Verdadeiro Irrelevante
3,0,eu quero muito assistir bacurau e era uma vez ...,1,Falso Positivo
5,1,vai sair uma versão de 4 horas de era uma vez ...,1,Verdadeiro Positivo
6,2,bacurau dá de 10 x 0 em era uma vez em hollywo...,0,Falso Irrelevante
9,2,quanto mais leio/ouço sobre era uma vez em ...,2,Verdadeiro Negativo
10,1,boa noite seguidores hoje a crítica vai para ...,1,Verdadeiro Positivo
11,0,era uma vez em hollywood ultrapassa a marca ...,0,Verdadeiro Irrelevante
12,0,dá rt se hoje você tá feliz tá animado tá an...,0,Verdadeiro Irrelevante


In [430]:
#Valores da frequência relativa da tabela concreta para realizar comparações
print(tabela_correta["Classificação"].value_counts(normalize=True)*100)

0    62.790698
1    27.242525
2     9.966777
Name: Classificação, dtype: float64


In [431]:
#Teste de qualidade do nosso classificador
#Porcentagens de nossas classificações
print(tabela_correta["Tipo"].value_counts(normalize=True)*100)

Verdadeiro Irrelevante    60.797342
Falso Irrelevante         19.269103
Verdadeiro Positivo       15.614618
Falso Positivo             2.990033
Verdadeiro Negativo        1.328904
Name: Tipo, dtype: float64


## Conclusão

Após usar o classificador Naive Bayes para diversos tweets da tabela Teste sobre "Era uma vez em Hollywood", novo filme do diretor Quentin Tarantino, foi possível observar uma porcentagem considerável de erros, que consistem em Falsos Irrelevantes (19.27%) e Falsos Positivos(2.99%), representando 22,25% do total. Isso ocorreu principalmente devido ao fato de que foi selecionado uma quantidade de tweets em que ocorria frequentimente a repetição de palavras, limitando a base de dados do classificador. É interessante observar que não obtivemos nenhum Falso Negativo, provavelmente, pela pequena quantidade de tweets "ruins". Porém, mesmo com esses erros, o classificador apresenta uma acurácia de 77,74%, consideravelmente alta. O maior índice de acertos está relacionado aos Verdadeiros irrelevantes, fato que ocorreu porque a maioria dos tweets nas bases de treinamento e teste de dados foram classificados como irrelevantes, aumentando sua probabilidade. Isso se dá graças ao grande número de usuários que estavam comentando a respeito do filme sem  ter o assistido. 

   Ao classificar manualmente os tweets que possuíam dupla negação ou sarcasmo, foi levado em consideração a conotação que eles sugeriam. Porém, a maioria era sobre a duração do filme, ou que ele era de difícil compreensão. Por isso, a maioria desses tweets foram classificados como de cunho negativo, e, como  o nosso classificador é um reflexo da classificação manual, a maior probabilidade é que um tweet com essas características seja classificado como Negativo.
   
   O nosso projeto deve continuar a ser financiado porque ele apresentou uma boa quantidade de acertos, mesmo utilizando uma pequena base de dados. Além disso, ele ajudará a empresa a ter um monitoramento dessa rede social de forma mais automatizada, sem que um funcionário precise ficar analisando cada tweet.Ainda, ele pode ficar melhor estruturado, tendo seu algoritmo executado em diferentes máquinas para aumentar a velocidade de classificação, como sugerido no site "https://machinelearningmastery.com/better-naive-bayes/"; além de um aumento na sua base de dados e categorias de classificação. Um exemplo seria utilizar uma ferramenta de monitoramento periódico, e, tornar a mensagem mais perceptível na time-line, caso for boa, ou chamando a atenção da equipe de Marketing, se for depreciativa. 
   
   O motivo pelo qual a base de treinamento não pode ser realimentada automaticamente utilizando o classificador, é que já existe um percentual de erro na classificação do banco de dados do sistema, e, ao colocar um novo tweet, ele pode sofrer essa influencia e ser classificado erroneamente também, alimentando a base de dados com um erro, e, portanto, prejudicando a eficácia do classificador. 
   